In [ ]:
# define rooms and items

# Game Room and its items
couch = {
    "name": "couch",
    "type": "furniture",
}

door_a = {
    "name": "door a",
    "type": "door",
}

piano = {
    "name": "piano",
    "type": "furniture",
}

game_room = {
    "name": "game room",
    "type": "room",
}

# Bedroom 1 and its items

bedroom_1 = {
    "name": "bedroom 1",
    "type": "room",
}

queen_bed = {
    "name": "queen bed",
    "type": "furniture",
}

door_b = {
    "name": "door b",
    "type": "door",
}

door_c = {
    "name": "door c",
    "type": "door",
}

# Bedroom 2 and its items

bedroom_2 = {
    "name": "bedroom 2",
    "type": "room",
}

double_bed = {
    "name": "double bed",
    "type": "furniture",
}

dresser = {
    "name": "dresser",
    "type": "furniture",
}

# Living Room and its items

living_room = {
    "name": "living room",
    "type": "room",
}

dining_table = {
    "name": "dining table",
    "type": "furniture",
}

door_d = {
    "name": "door d",
    "type": "door",
}

# outside : Destination

outside = {
  "name": "outside"
}

# keys linked to its target rooms

key_a = {
    "name": "key for door a",
    "type": "key",
    "target": door_a,
}

key_b = {
    "name": "key for door b",
    "type": "key",
    "target": door_b
}

key_c = {
    "name": "key for door c",
    "type": "key",
    "target": door_c
}

key_d = {
    "name": "key for door d",
    "type": "key",
    "target": door_d
}

all_rooms = [game_room, bedroom_1, bedroom_2, living_room, outside]

all_doors = [door_a, door_b, door_c, door_d]

# define which items/rooms are related

object_relations = {
    "game room": [couch, piano, door_a],
    "bedroom 1": [queen_bed, door_a, door_b, door_c],
    "bedroom 2": [double_bed, dresser, door_b],
    "living room": [door_c, dining_table, door_d],
    "piano": [key_a],
    "queen bed": [key_b],
    "double bed": [key_d, key_c], #Moved key from dresser to double bed so that it has both keys
    "outside": [door_d],
    "door a": [game_room, bedroom_1],
    "door b": [bedroom_1, bedroom_2],
    "door c": [bedroom_1, living_room],
    "door d": [living_room, outside]
}

# define game state. Do not directly change this dict.
# Instead, when a new game starts, make a copy of this
# dict and use the copy to store gameplay state. This
# way you can replay the game multiple times.

INIT_GAME_STATE = {
    "current_room": game_room,
    "keys_collected": [],
    "target_room": outside
}

In [ ]:
def linebreak():
    """
    Print a line break
    """
    print("\n\n")

def start_game():
    """
    Start the game
    """
    print("You wake up on a couch and find yourself in a strange house with no windows which you have never been to before. You don't remember why you are here and what had happened before. You feel some unknown danger is approaching and you must get out of the house, NOW!")
    play_room(game_state["current_room"])

def play_room(room):
    """
    Play a room. First check if the room being played is the target room.
    If it is, the game will end with success. Otherwise, let player either
    explore (list all items in this room) or examine an item found here.
    """
    game_state["current_room"] = room
    if(game_state["current_room"] == game_state["target_room"]):
        print("Congrats! You escaped the room!")
    else:
        print("You are now in " + room["name"])
        intended_action = input("What would you like to do? Type 'explore' or 'examine'? ").strip().lower()
        if intended_action == "explore":
            explore_room(room)
            play_room(room)
        elif intended_action == "examine":
            examine_item(input("What would you like to examine? ").strip())
        else:
            print("Not sure what you mean. Type 'explore' or 'examine'.")
            play_room(room)
        linebreak()

def explore_room(room):
    """
    Explore a room. List all items belonging to this room.
    """
    items = [i["name"] for i in object_relations[room["name"]]]
    print("You are exploring the room. This is " + room["name"] + ". You found '" + "', '".join(items)+"'") #Made the message look better

def get_next_room_of_door(door, current_room):
    """
    From object_relations, find the two rooms connected to the given door.
    Return the room that is not the current_room.
    """
    connected_rooms = object_relations[door["name"]]
    for room in connected_rooms:
        if(not current_room == room):
            return room

def examine_item(item_name):
    """
    Examine an item which can be a door or furniture.
    First make sure the intended item belongs to the current room.
    Then check if the item is a door. Tell player if key hasn't been
    collected yet. Otherwise ask player if they want to go to the next
    room. If the item is not a door, then check if it contains keys.
    Collect the key if found and update the game state. At the end,
    play either the current or the next room depending on the game state
    to keep playing.
    """
    current_room = game_state["current_room"]
    next_room = ""
    output = None

    for item in object_relations[current_room["name"]]:
        if(item["name"] == item_name):
            output = "You examined '" + item_name + "'. " #added quotes for better understanding of items being examined
            if(item["type"] == "door"):
                have_key = False
                key_name = ""
                for key in game_state["keys_collected"]:
                    if(key["target"] == item):
                        have_key = True
                        key_name = key["name"]
                if(have_key):
                    output += f"You unlock it with '{key_name}' you have." #Adding the key name which will be used to unlock the door
                    next_room = get_next_room_of_door(item, current_room)
                else:
                    #Added a if else block to display list of all keys the user has.
                    if len(game_state["keys_collected"]) > 0:
                      output += "It is locked but you don't have the key. You only have '"+"', '".join([key["name"] for key in game_state["keys_collected"]])+"'" #Used list comprehension to display the list of keys the user already has
                    else:
                      output += "It is locked but you don't have any keys."
            else:
                if(item["name"] in object_relations and len(object_relations[item["name"]])>0):
                    item_found = object_relations[item["name"]].pop()
                    game_state["keys_collected"].append(item_found)
                    output += "You found '" + item_found["name"] + "'. You can try to examine '"+item_found["target"]["name"]+"' to move further in the game"
                else:
                    output += "There isn't anything interesting about it, Sorry. Please try to examine a different item and try your luck." #Modify message for better understanding
            print(output)
            break

    if(output is None):
        print("The item you requested is not found in the current room.")

    if(next_room and input(f"Do you want to go to '{next_room['name']}'? Enter 'yes' or 'no' ").strip().lower() == 'yes'): #added room name to specify which room the user will be going to instead of just mentioning next room and also fixed the typo
        play_room(next_room)
    else:
        play_room(current_room)


game_state = INIT_GAME_STATE.copy()

start_game()

You wake up on a couch and find yourself in a strange house with no windows which you have never been to before. You don't remember why you are here and what had happened before. You feel some unknown danger is approaching and you must get out of the house, NOW!
You are now in game room
What would you like to do? Type 'explore' or 'examine'? explore
You are exploring the room. This is game room. You found 'couch', piano', door a'
You are now in game room
What would you like to do? Type 'explore' or 'examine'? examine
What would you like to examine? couch
You examined 'couch'. There isn't anything interesting about it, Sorry. Please try to examine a different item and try your luck.
You are now in game room
What would you like to do? Type 'explore' or 'examine'? examine
What would you like to examine? piano
You examined 'piano'. You found 'key for door a'. You can try to examine 'door a' to move further in the game
You are now in game room
What would you like to do? Type 'explore' or 'e